In [2]:
import pandas as pd
import numpy as np
import opendatasets as od
import time
import math
import datetime
from torchsummary import summary
import torch.nn as nn
pd.options.mode.chained_assignment = None
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
tqdm.pandas()

In [2]:
dataset_url = 'https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction'
od.download(dataset_url)

Skipping, found downloaded files in "./new-york-city-taxi-fare-prediction" (use force=True to force download)


In [14]:
df = pd.read_csv('./new-york-city-taxi-fare-prediction/train.csv')

In [18]:
date_cap = datetime.datetime(2015, 12, 31)
date_cap < datetime.datetime.now()


True

In [ ]:
def ToTimeStamp(row):
    return pd.Timestamp(row)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
#df['month_formatted'] = df['pickup_datetime'].progress_apply(lambda row: pd.Period(row, freq='D'))

#2010-04-18


In [8]:
df['pickup_datetime']

0           2009-06-15 17:26:21 UTC
1           2010-01-05 16:52:16 UTC
2           2011-08-18 00:35:00 UTC
3           2012-04-21 04:30:42 UTC
4           2010-03-09 07:51:00 UTC
                     ...           
55423851    2014-03-15 03:28:00 UTC
55423852    2009-03-24 20:46:20 UTC
55423853    2011-04-02 22:04:24 UTC
55423854    2011-10-26 05:57:51 UTC
55423855    2014-12-12 11:33:00 UTC
Name: pickup_datetime, Length: 55423856, dtype: object

In [ ]:
df=df[~(df['month_formatted']<'2015-12-31')]

In [30]:
print(df['pickup_datetime'].max())
print(df['pickup_datetime'].min())

2015-06-30 23:59:54 UTC
2009-01-01 00:00:27 UTC


True

In [125]:
## Running this cell is really fucking slow, but it works.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

def havardsine_distance(lat1, long1, lat2, long2):
    r = 6371

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)

    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(long2 - long1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = (r * c) # convert to kilometer
    return d

df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

airport_cords = [
    [40.691547, -74.180202], # NEWARK
    [40.773281, -73.869845], # LAGUARDIA
]

# When I wrote this code, only God and I knew what was going on. Today, only God knows.
def close_to_airport(values):
    pickup_latitude = values['pickup_latitude']
    pickup_longitude = values['pickup_longitude']
    dropoff_latitude = values['dropoff_latitude']
    dropoff_longitude = values['dropoff_longitude']

    # JFK is special, it needs a bigger radius
    jfkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.645042, -73.786928) <= 1.00
    jfkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.645042, -73.786928) <= 1.00

    # Newark is a small airport but their parking log is BIG.
    newarkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.691547, -74.180202) <= 1.00
    newarkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.691547, -74.180202) <= 1.00

    # Third largest, it's an abstract airport with a stretched out U shaped parking place
    # LaGuardia
    laguardiapickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.773855, -73.871712) <= 0.50
    laguardiadropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.773855, -73.871712) <= 0.50

    return 1 if jfkpickup or jfkdropoff or newarkpickup or newarkdropoff or laguardiapickup or laguardiadropoff else 0

df['is_by_airport'] = df.apply(close_to_airport, axis=1)

In [126]:
my_time = df['pickup_datetime'][0]

In [127]:
df['EDTdate'] = df['pickup_datetime'] - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour'] < 12, 0, 1)

In [128]:
df['Weekday'] = pd.to_numeric(df['EDTdate'].dt.strftime("%w")) # an int between 0 - 6, representing the weekdays
df['Month'] = pd.to_numeric(df['EDTdate'].dt.strftime("%m")) # an int between 0 - 6, representing the weekdays

#Formatting this correctly, we can use the dates to get the historical weather data for the dates
df['FullDate'] = df['EDTdate'].dt.strftime('%x')
#print(df['FullDate'].min())
#print(df['FullDate'].max())
print(df['FullDate'].unique())

['04/19/10' '04/17/10' '04/11/10' '04/16/10' '04/22/10' '04/23/10'
 '04/15/10' '04/20/10' '04/21/10' '04/14/10' '04/13/10' '04/12/10'
 '04/24/10' '04/18/10']


In [129]:
'''
api_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"
new_york_middle_lat = ""
new_york_middle_long = ""
unique = np.sort(df['FullDate'].unique())
start = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())
end = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())
url = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"

print(url)
'''
# Historical weather data, not working

'\napi_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"\nnew_york_middle_lat = ""\nnew_york_middle_long = ""\nunique = np.sort(df[\'FullDate\'].unique())\nstart = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())\nend = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())\nurl = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"\n\nprint(url)\n'

In [237]:
# lets have some weather-y fuuuuuuuuuuuun-fuuuuuun-function
# see this for doc https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
weather_data = pd.read_csv('nyc-weather-data.csv')
weather_labels = ['SNWD', 'SNOW', 'AWND'] #snow depth, did it snow
for index, item in enumerate(weather_labels):
    weather_data.loc[weather_data[item] <= -9999, item] = 0 # -9999 means that there is no data

# df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

weather_data['DATE'] = weather_data.apply(lambda row: datetime.datetime.strptime(str(row.DATE), '%Y%m%d').strftime('%m/%d/%y'), axis=1)
weather_data = weather_data[['DATE','PRCP','SNWD','SNOW','TMAX','TMIN','AWND']]

weather_data['TMIN'] = (weather_data['TMIN'] / 10) # the API returns degrees in an old format. We therefore convert the value to a tenth of its own form. Check the docu for this.
weather_data['TMAX'] = (weather_data['TMAX'] / 10)
weather_data['PRCP'] = weather_data['PRCP'] / 10

weather_data['AWND'] = weather_data['AWND'] / 2.237 # meters per second

weather_data['DATE'].max()

#df.rename(columns={'FullDate':'DATE'}, inplace=True)

#merged_df = pd.merge(df, weather_data, how="left",on="DATE")

'12/31/14'

In [164]:
network_data = merged_df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km', 'passenger_count', 'is_by_airport', 'Hour', 'AMorPM', 'Weekday', 'PRCP', 'TMAX', 'TMIN', 'AWND', 'fare_amount']]

network_data['pickup_longitude'] = network_data.apply(lambda row: abs(row.pickup_longitude), axis=1)

network_data['dropoff_longitude'] = network_data.apply(lambda row: abs(row.pickup_longitude), axis=1)

network_data.iloc[:,0:-1] = network_data.iloc[:,0:-1].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

prediction = network_data['fare_amount']

network_data.drop('fare_amount', axis=1, inplace=True)
X = network_data.values

y = prediction.values

(120000,)


In [165]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

In [194]:
X = torch.tensor(train_X, dtype=torch.float32)
y = torch.tensor(train_y, dtype=torch.long)
test_X_tensor = torch.Tensor(test_X)
test_y_tensor = torch.LongTensor(test_y)

train_ds = torch.utils.data.TensorDataset(X, y)
test_ds = torch.utils.data.TensorDataset(test_X_tensor, test_y_tensor)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)

In [230]:
class NYCTaxiPredictor(nn.Module):
    def __init__(self):
        super(NYCTaxiPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.6),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(0.6),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, x):
        return self.layers(x)


model = NYCTaxiPredictor()
summary(model, tuple([14]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             960
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
       BatchNorm1d-4                   [-1, 64]             128
           Dropout-5                   [-1, 64]               0
              ReLU-6                   [-1, 64]               0
            Linear-7                  [-1, 128]           8,320
       BatchNorm1d-8                  [-1, 128]             256
           Dropout-9                  [-1, 128]               0
             ReLU-10                  [-1, 128]               0
           Linear-11                  [-1, 128]          16,512
       LogSoftmax-12                  [-1, 128]               0
Total params: 30,336
Trainable params: 30,336
Non-trainable params: 0
---------------------------------

In [231]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()
loss_values = []
for epochs in range(100):
    for data in train_dl:
        X, y = data
        optimizer.zero_grad()
        output = model.forward(X)
        #print(output.shape, y.shape)
        loss = nn.functional.nll_loss(output, y)
        #loss_values.append(loss.item())
        #print(loss.item())
        loss.backward()
        optimizer.step()
    print(loss.data, epochs)

tensor(2.1231) 0
tensor(1.9682) 1
tensor(1.9641) 2
tensor(2.1875) 3
tensor(2.0390) 4
tensor(2.1349) 5
tensor(2.1541) 6
tensor(1.8245) 7
tensor(2.0170) 8
tensor(2.0522) 9
tensor(1.9900) 10
tensor(1.8851) 11
tensor(1.8808) 12
tensor(2.1794) 13
tensor(2.0594) 14
tensor(2.0441) 15
tensor(2.0263) 16
tensor(1.8839) 17
tensor(1.9900) 18
tensor(2.0724) 19
tensor(1.7681) 20
tensor(1.9962) 21
tensor(2.1896) 22
tensor(1.8728) 23
tensor(2.0756) 24
tensor(1.9152) 25
tensor(1.9879) 26
tensor(1.8857) 27
tensor(1.8706) 28
tensor(1.8354) 29
tensor(1.8824) 30
tensor(2.1126) 31
tensor(2.1036) 32
tensor(2.0349) 33
tensor(1.9256) 34
tensor(1.9295) 35
tensor(1.6889) 36
tensor(2.1701) 37
tensor(1.9238) 38
tensor(1.7796) 39
tensor(1.7728) 40
tensor(2.0054) 41
tensor(1.9527) 42
tensor(1.8442) 43
tensor(1.9331) 44
tensor(2.2847) 45
tensor(2.0903) 46
tensor(1.8294) 47
tensor(1.9824) 48
tensor(1.8595) 49
tensor(1.9283) 50
tensor(1.9118) 51
tensor(1.8806) 52
tensor(2.3116) 53
tensor(1.9915) 54
tensor(1.8433) 55
te

In [232]:
total = 0
correct = 0
with torch.no_grad(): # no need to keep track of the gradient. The backward() is doing it
  for data in test_dl:
    X, y = data
    output = model(X)
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print("Accuracy:", round(correct/total, 3) * 100)

Accuracy: 33.2
